# INIT

In [22]:
import os, re
os.chdir(os.path.realpath(os.path.curdir + '/..'))
print(os.path.realpath(os.path.curdir))

C:\Workspace\src


In [2]:
import random, string, arrow, requests, json
from time import sleep

In [3]:
rint = lambda x: random.randint(0,x)

In [4]:
import threading

# End to end test

In [6]:
from birdsong import *

In [7]:
testingDataset = 'Testing'

In [8]:
tagPaths = [
     '.'.join([testingDataset, ''.join([random.choice(string.ascii_uppercase) for i in range(6)])])
    for _ in range(3)
]
print(tagPaths)

['Testing.UMNWEQ', 'Testing.JGTLOM', 'Testing.ANXAWN']


## Get view setup

### Get view names 

In [9]:
with CanaryView() as view:
    for node in view.browseNodes():
        print(node)

CS-Surface61
Test Model


### Get datasets in view

In [10]:
with CanaryView() as view:
    for node in view.browseNodes('CS-Surface61'):
        print(node)

ProCHRO
Testing
Testing2
{Diagnostics}


### Autoresolve for testing

In [11]:
def getTestingNode():
    with CanaryView() as view:
        for node in view.browseNodes():
            for dataset in view.browseNodes(node):
                if dataset == 'Testing':
                    return node
mainView = getTestingNode()
print(mainView)

CS-Surface61


## Inject data for one tag

In [12]:
tvqDict = {
    tagPaths[0]: [
        ('2019-10-01 01:23:45', 1.23),
        ('2019-10-01 02:34:56', 2.34,192),
        Tvq('2019-10-01 03:45:01', 3.45),
        Tvq('2019-10-01 04:56:12', 4.56,192),
    ]
}

with CanarySender() as send:
    send.storeData(tvqDict)

## Retrieve data for one tag

In [12]:
tagPath = mainView + '.' + tagPaths[0]
print(tagPath)

CS-Surface61.Testing.GPPMKK


In [13]:
# Get the default value (most recent - will be the No Data value)
with CanaryView() as view:
    for value in view.getTagData(tagPath):
        print(value)

{'timestamp': '2019-10-01T04:56:12.0000001-07:00', 'value': None}


In [14]:
# Get multiple values
with CanaryView() as view:
    for value in view.getTagData(tagPath, 
                                 startTime='2019-10-01T00:00:00-0700',
                                 endTime='2019-10-01T03:00-0700'):
        print(value)

{'timestamp': '2019-10-01T01:23:45.0000000-07:00', 'value': 1.23}
{'timestamp': '2019-10-01T02:34:56.0000000-07:00', 'value': 2.34}


## Inject data for multiple tags

In [15]:
tvqDict = {
    tagPaths[1]: [
        ('2019-10-01 01:11:11', 1.11),
        ('2019-10-01 02:22:22', 2.22, 192),
        Tvq('2019-10-01 03:33:33', 3.33),
        Tvq('2019-10-01 04:44:44', 4.44, 192),
    ],
    tagPaths[2]: [
        ('2019-10-01 01:00:00', 1),
        ('2019-10-01 02:00:00', 2, 192),
        Tvq('2019-10-01 03:00:00', 3),
        Tvq('2019-10-01 04:00:00', 4, 192),
    ]
}

with CanarySender() as send:
    send.storeData(tvqDict)

## Retrieve data for multiple tags

In [16]:
tagList = [mainView + '.' + tagPath for tagPath in tagPaths[1:3]]
print(tagList)

['CS-Surface61.Testing.RDMVUH', 'CS-Surface61.Testing.FMBNPQ']


In [17]:
# Get the default value (most recent - will be the No Data value)
with CanaryView() as view:
    for tagPath, values in view.getTagData(tagList):
        print(tagPath)
        for value in values:
            print('\t%r' % value)

CS-Surface61.Testing.RDMVUH
	{'timestamp': '2019-10-01T04:44:44.0000001-07:00', 'value': None}
CS-Surface61.Testing.FMBNPQ
	{'timestamp': '2019-10-01T04:00:00.0000001-07:00', 'value': None}


In [18]:
# Get the values for each tag between given dates
with CanaryView() as view:
    for tagPath, values in view.getTagData(tagList, 
                                           start='2019-10-01T00:00:00-0700',
                                           end='2019-10-01T03:00-0700'):
        print(tagPath)
        for value in values:
            print('\t%r' % value)

CS-Surface61.Testing.RDMVUH
	{'timestamp': '2019-10-01T01:11:11.0000000-07:00', 'value': 1.11}
	{'timestamp': '2019-10-01T02:22:22.0000000-07:00', 'value': 2.22}
CS-Surface61.Testing.FMBNPQ
	{'timestamp': '2019-10-01T01:00:00.0000000-07:00', 'value': 1}
	{'timestamp': '2019-10-01T02:00:00.0000000-07:00', 'value': 2}


## Inject and retrieve live data

In [19]:
start = arrow.get('2019-10-02 12:00:00-0700')

In [20]:
start.shift(seconds=5)

<Arrow [2019-10-02T12:00:05-07:00]>

In [21]:
testDuration = 10
testInterval = 2

In [22]:
def generateData(tagPaths=tagPaths, duration=testDuration, interval=testInterval):
    
    with CanarySender() as send:
        for step in range(testDuration//interval):

            tvqDict = {}
            
            for ix,tagPath in enumerate(tagPaths):
                tvqDict[tagPath] = [ Tvq(arrow.get().isoformat(), 100*step + ix) ]

            send.storeData(tvqDict)
            
            sleep(interval)

### One Tag

In [23]:
t1 = threading.Thread(target=generateData, args=([tagPaths[0]],))
t1.start()

In [24]:
stepTime = 3

with CanaryView() as view:
    for step in range( (testDuration//stepTime) + 2):
        print('Update %d' % step)
        for value in view.getLiveData(mainView + '.' + tagPaths[0]):
            print(value)
        sleep(stepTime)

Update 0
{'timestamp': '2019-10-01T04:56:12.0000001-07:00', 'value': None}
Update 1
{'timestamp': '2019-10-28T13:06:21.8470740-07:00', 'value': 0}
{'timestamp': '2019-10-28T13:06:23.8692130-07:00', 'value': 100}
Update 2
{'timestamp': '2019-10-28T13:06:25.8798020-07:00', 'value': 200}
Update 3
{'timestamp': '2019-10-28T13:06:27.8900810-07:00', 'value': 300}
{'timestamp': '2019-10-28T13:06:29.8937230-07:00', 'value': 400}
Update 4
{'timestamp': '2019-10-28T13:06:29.8937231-07:00', 'value': None}


### Multiple tags

In [25]:
# Only update the tags after the first
t2= threading.Thread(target=generateData, args=(tagPaths[1:],))
t2.start()

In [26]:
# Check all tags so that we can see the first _not_ get updated in later calls

viewQualifiedTagPaths = [mainView + '.' + tagPath for tagPath in tagPaths]
stepTime = 3

with CanaryView() as view:
    for step in range( (testDuration//stepTime)+2):
        print('Update %d' % step)
        for tagPath,values in view.getLiveData(viewQualifiedTagPaths):
            print('\t%s' % tagPath)
            for value in values:
                print('\t\t%r' % value)
        sleep(stepTime)

Update 0
	CS-Surface61.Testing.GPPMKK
		{'timestamp': '2019-10-28T13:06:29.8937231-07:00', 'value': None}
	CS-Surface61.Testing.RDMVUH
		{'timestamp': '2019-10-01T04:44:44.0000001-07:00', 'value': None}
	CS-Surface61.Testing.FMBNPQ
		{'timestamp': '2019-10-01T04:00:00.0000001-07:00', 'value': None}
Update 1
	CS-Surface61.Testing.RDMVUH
		{'timestamp': '2019-10-28T13:06:36.9281550-07:00', 'value': 0}
		{'timestamp': '2019-10-28T13:06:38.9378920-07:00', 'value': 100}
	CS-Surface61.Testing.FMBNPQ
		{'timestamp': '2019-10-28T13:06:36.9281550-07:00', 'value': 1}
		{'timestamp': '2019-10-28T13:06:38.9378920-07:00', 'value': 101}
Update 2
	CS-Surface61.Testing.RDMVUH
		{'timestamp': '2019-10-28T13:06:40.9497650-07:00', 'value': 200}
	CS-Surface61.Testing.FMBNPQ
		{'timestamp': '2019-10-28T13:06:40.9497650-07:00', 'value': 201}
Update 3
	CS-Surface61.Testing.RDMVUH
		{'timestamp': '2019-10-28T13:06:42.9556010-07:00', 'value': 300}
		{'timestamp': '2019-10-28T13:06:44.9681840-07:00', 'value': 4

# End Automated Testing

In [14]:
raise KeyboardInterrupt

KeyboardInterrupt: 